In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

import os
from dotenv import load_dotenv
load_dotenv() # load all the environment variables

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

loader = TextLoader("longSpeech.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=700,chunk_overlap = 20)
docs = text_splitter.split_documents(documents=documents)

docs

In [7]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

db = FAISS.from_documents(docs,embedding=embeddings)
db

In [10]:
# query

query1 = "What is the speaker trying to say?"
query2 = "Why is the speaker not satisfied?"

docs_result = db.similarity_search(query2)
docs_result[0].page_content

'There are those who are asking the devotees of civil rights, "When will you be satisfied?" We can never be satisfied as long as the Negro is the victim of the unspeakable horrors of police brutality. We can never be satisfied as long as our bodies, heavy with the fatigue of travel, cannot gain lodging in the motels of the highways and the hotels of the cities. We cannot be satisfied as long as the negro\'s basic mobility is from a smaller ghetto to a larger one. We can never be satisfied as long as our children are stripped of their self-hood and robbed of their dignity by signs stating: "For Whites Only."We cannot be satisfied as long as a Negro in Mississippi cannot vote and a Negro in New York believes he has nothing for which to vote. No, no, we are not satisfied, and we will not be satisfied until "justice rolls down like waters, and righteousness like a mighty stream."'

In [13]:
# retriever

retriever = db.as_retriever()
retriever_result = retriever.invoke(query2)
retriever_result[0].page_content

'There are those who are asking the devotees of civil rights, "When will you be satisfied?" We can never be satisfied as long as the Negro is the victim of the unspeakable horrors of police brutality. We can never be satisfied as long as our bodies, heavy with the fatigue of travel, cannot gain lodging in the motels of the highways and the hotels of the cities. We cannot be satisfied as long as the negro\'s basic mobility is from a smaller ghetto to a larger one. We can never be satisfied as long as our children are stripped of their self-hood and robbed of their dignity by signs stating: "For Whites Only."We cannot be satisfied as long as a Negro in Mississippi cannot vote and a Negro in New York believes he has nothing for which to vote. No, no, we are not satisfied, and we will not be satisfied until "justice rolls down like waters, and righteousness like a mighty stream."'

In [14]:
# Similarity search with score

docs_and_score = db.similarity_search_with_score(query2)
docs_and_score

[(Document(id='5412cb8d-10c4-40b9-95f5-2e8acaf1ed0d', metadata={'source': 'longSpeech.txt'}, page_content='There are those who are asking the devotees of civil rights, "When will you be satisfied?" We can never be satisfied as long as the Negro is the victim of the unspeakable horrors of police brutality. We can never be satisfied as long as our bodies, heavy with the fatigue of travel, cannot gain lodging in the motels of the highways and the hotels of the cities. We cannot be satisfied as long as the negro\'s basic mobility is from a smaller ghetto to a larger one. We can never be satisfied as long as our children are stripped of their self-hood and robbed of their dignity by signs stating: "For Whites Only."We cannot be satisfied as long as a Negro in Mississippi cannot vote and a Negro in New York believes he has nothing for which to vote. No, no, we are not satisfied, and we will not be satisfied until "justice rolls down like waters, and righteousness like a mighty stream."'),
  

In [16]:
embeddings_vector = embeddings.embed_query(query2)
resultByVector = db.similarity_search_by_vector(embeddings_vector)
resultByVector[0].page_content

'There are those who are asking the devotees of civil rights, "When will you be satisfied?" We can never be satisfied as long as the Negro is the victim of the unspeakable horrors of police brutality. We can never be satisfied as long as our bodies, heavy with the fatigue of travel, cannot gain lodging in the motels of the highways and the hotels of the cities. We cannot be satisfied as long as the negro\'s basic mobility is from a smaller ghetto to a larger one. We can never be satisfied as long as our children are stripped of their self-hood and robbed of their dignity by signs stating: "For Whites Only."We cannot be satisfied as long as a Negro in Mississippi cannot vote and a Negro in New York believes he has nothing for which to vote. No, no, we are not satisfied, and we will not be satisfied until "justice rolls down like waters, and righteousness like a mighty stream."'

In [17]:
# Saving and Loading

db.save_local("faiss_index")

In [19]:
new_db = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)

In [20]:
docs_result_local = new_db.similarity_search(query2)
docs_result_local[0].page_content

'There are those who are asking the devotees of civil rights, "When will you be satisfied?" We can never be satisfied as long as the Negro is the victim of the unspeakable horrors of police brutality. We can never be satisfied as long as our bodies, heavy with the fatigue of travel, cannot gain lodging in the motels of the highways and the hotels of the cities. We cannot be satisfied as long as the negro\'s basic mobility is from a smaller ghetto to a larger one. We can never be satisfied as long as our children are stripped of their self-hood and robbed of their dignity by signs stating: "For Whites Only."We cannot be satisfied as long as a Negro in Mississippi cannot vote and a Negro in New York believes he has nothing for which to vote. No, no, we are not satisfied, and we will not be satisfied until "justice rolls down like waters, and righteousness like a mighty stream."'